In [ ]:
!pip install langsmith > /dev/null
!pip install langchain > /dev/null
!pip install openai > /dev/null
!pip install python-dotenv > /dev/null

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm.predict("How are you?")

### Using Langsmith

In [ ]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")
llm.predict("How many people live in USA?", callbacks=[tracer])

In [ ]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    llm.predict("How many people live in USA?")

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["my-llm-tag"])
prompt = PromptTemplate.from_template("Say {input}")
chain = LLMChain(llm=llm, prompt=prompt, tags=["one-tag", "another-tag"])

chain("Hello, World!", tags=["shared-tags"])

In [ ]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("my_group_name") as group_manager:
    pass

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
with trace_as_chain_group("my_group") as group_manager:
    chain.run(question="What is your name?", callbacks=group_manager)
    chain.run(question="What is your quest?", callbacks=group_manager)
    chain.run(question="What is your favorite color?", callbacks=group_manager)

In [ ]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

In [ ]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs


In [ ]:
for run in todays_runs:
    print(run)

### Metadata
Add metadata to filter runs later, for example for making an A/B test


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chat_model = ChatOpenAI()
chain = LLMChain.from_string(llm=chat_model, template="What's the answer to {input}?")

chain({"input": "What is the meaning of life?"}, metadata={"source": "korean"})

In [ ]:
from langsmith import Client

client = Client()
runs = list(client.list_runs(
    filter='has(metadata, \'{"source": "korean"}\')',
))
print(list(runs))